## Importing the required libraries

In [1]:
import pandas as pd
import math
import matplotlib.pyplot as plt
import seaborn as sns
from textblob import TextBlob
import re
import string
import random
import seaborn as sns
import warnings
import time
import category_encoders as ce
import nltk
import numpy as np
from nltk.corpus import stopwords

%run Functions.ipynb
from sklearn.svm import SVC
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report,accuracy_score
from xgboost import XGBClassifier
from sklearn import preprocessing
from sklearn.decomposition import PCA

from sklearn.metrics import f1_score, recall_score, precision_score
from sklearn.preprocessing import Binarizer
from sklearn.linear_model import SGDClassifier
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder, FunctionTransformer, Binarizer,LabelEncoder, LabelBinarizer
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegressionCV, Perceptron, LogisticRegression
from sklearn.pipeline import Pipeline
pd.set_option('display.max_columns', None)

from sklearn.preprocessing import StandardScaler
from category_encoders import TargetEncoder

## Reading and Extracting data from .csv files

In [2]:
key_train = pd.read_csv('train_features_1.csv')
key_test = pd.read_csv('test_features_1.csv')

#Uso train y test filtrado por axel para poder usar datos de location
train_tweets = pd.read_csv('FE_1_train.csv')
test_tweets = pd.read_csv('FE_1_test.csv')

In [3]:
def load_dict_from_file(name):
    f = open(name,'r')
    data=f.read()
    f.close()
    return eval(data)

dict_0 = load_dict_from_file("dict_0.txt")
dict_1 = load_dict_from_file("dict_1.txt")
listaPalabras = []

archivo = open("NW.txt", "r")
for linea in archivo.readlines():
    linea = linea.rstrip('\n') 
    listaPalabras.append(linea)
archivo.close()



prob_words(train_tweets,test_tweets,listaPalabras,dict_0,dict_1)

In [4]:
train_tweets["word_count"] =key_train["word_count"]
train_tweets["unique_word_count"] =key_train["unique_word_count"]
train_tweets["url_count"] = key_train["url_count"]
train_tweets["mean_word_length"] = key_train["mean_word_length"]

train_tweets\
.drop(columns=["id",'complete_text','negativity','sigmoidic_negativity','neg_to_label','Unnamed: 0','location'], axis=1,inplace = True)

test_tweets["word_count"] =key_test["word_count"]
test_tweets["unique_word_count"] =key_test["unique_word_count"]
test_tweets["url_count"] = key_test["url_count"]
test_tweets["mean_word_length"] = key_test["mean_word_length"]

ids = test_tweets["id"]
test_tweets\
.drop(columns=["id",'complete_text','negativity','sigmoidic_negativity','neg_to_label','Unnamed: 0','location'], axis=1,inplace = True)

train_tweets["keyword"] = key_train["keyword"]
test_tweets["keyword"] = key_test["keyword"]
train_tweets["text_filtrado"] = key_train["text"]
test_tweets["text_filtrado"] = key_test["text"]

train_tweets.fillna("NaN", inplace= True)
test_tweets.fillna("NaN", inplace= True)
train_tweets['keyword'] = train_tweets['keyword'].str.replace('%',' ')
train_tweets['keyword'] = train_tweets['keyword'].str.replace('2','')
train_tweets['keyword'] = train_tweets['keyword'].str.replace('0','')

test_tweets['keyword'] = test_tweets['keyword'].str.replace('%',' ')
test_tweets['keyword'] = test_tweets['keyword'].str.replace('2','')
test_tweets['keyword'] = test_tweets['keyword'].str.replace('0','')

keyword_in_text_2(train_tweets)
keyword_in_text_2(test_tweets)
Has(train_tweets)
Has(test_tweets)

c:\users\hal9000\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.


In [5]:
import category_encoders as ce

ct =ce.TargetEncoder()
train_tweets["key_endoder"]= ct.fit_transform(train_tweets["keyword"],train_tweets["target"])
test_tweets["key_endoder"]= ct.transform(test_tweets["keyword"])

le = LabelEncoder()
train_tweets["Label_key"] = le.fit_transform(train_tweets["keyword"])
test_tweets["Label_key"] = le.transform(test_tweets["keyword"])

y = train_tweets['target']

train_tweets.drop(columns=['target'], axis=1,inplace = True)

In [6]:
bi = ce.BinaryEncoder(cols = ["continent","country","city","keyword"],handle_unknown="value")
train_tweets = bi.fit_transform(train_tweets)
test_tweets = bi.transform(test_tweets)


oh=ce.OneHotEncoder(cols =["Label_key"],handle_unknown="ignore")
train_tweets = oh.fit_transform(train_tweets)
test_tweets = oh.transform(test_tweets)

In [7]:
def truncate(number, digits) -> float:
    stepper = 10.0 ** digits
    return math.trunc(stepper * number) / stepper

In [8]:
train_tweets["mean_word_length"] =train_tweets["mean_word_length"].map(lambda x: truncate(x,3))
test_tweets["mean_word_length"] =test_tweets["mean_word_length"].map(lambda x: truncate(x,3))
#train_tweets["Puntaje_ser_1"] =train_tweets["Puntaje_ser_1"].map(lambda x: truncate(x,3))
#test_tweets["Puntaje_ser_0"] =test_tweets["Puntaje_ser_0"].map(lambda x: truncate(x,3))

In [9]:
train_tweets.replace({True: 1, False: 0})

,keyword_0,keyword_1,keyword_2,keyword_3,keyword_4,keyword_5,keyword_6,keyword_7,keyword_8,text,continent_0,continent_1,continent_2,continent_3,continent_4,country_0,country_1,country_2,country_3,country_4,country_5,country_6,country_7,city_0,city_1,city_2,city_3,city_4,city_5,city_6,city_7,city_8,city_9,city_10,hasNum,hasLink,hasUser,hasHashtag,exclamacion,interrogacion,mayusculas,longitud,Puntaje_ser_1,Puntaje_ser_0,word_count,unique_word_count,url_count,mean_word_length,text_filtrado,contains_keyword,Tiene_key_impor,Tiene_key_no_impor,key_endoder,Label_key
0,0,0,0,0,0,0,0,0,1,Our Deeds are the Reason of this #earthquake M...,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,10,71,1.512000,2.294000,13,13,0,4.384,our deeds are reason earthquake may allah forg...,0,0,0,0.688525,0
1,0,0,0,0,0,0,0,0,1,Forest fire near La Ronge Sask. Canada,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,5,40,1.697714,2.108286,7,7,0,4.571,forest fire near la ronge sask canada,0,0,0,0.688525,0
2,0,0,0,0,0,0,0,0,1,All residents asked to 'shelter in place' are ...,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,2,135,1.612000,2.194000,22,20,0,5.090,residents asked shelter in place are being not...,0,0,0,0.688525,0
3,0,0,0,0,0,0,0,0,1,"13,000 people receive #wildfires evacuation or...",0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,1,0,0,1,0,0,1,67,2.966851,4.839149,9,8,0,7.125,people receive wildfires evacuation orders in ...,0,0,0,0.688525,0
4,0,0,0,0,0,0,0,0,1,Just got sent this photo from Ruby #Alaska as ...,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,3,90,2.766851,4.039149,17,15,0,4.500,just got sent photo from ruby alaska smoke fro...,0,0,0,0.688525,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7608,0,0,0,0,0,0,0,0,1,Two giant cranes holding a bridge collapse int...,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,1,1,0,0,0,0,7,85,1.732000,2.074000,11,11,1,6.636,two giant cranes holding bridge collapse into ...,0,0,0,0.688525,0
7609,0,0,0,0,0,0,0,0,1,@aria_ahrary @TheTawniest The out of control w...,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,6,127,1.857714,2.948286,20,17,0,5.300,ariaahrary thetawniest out control wild fires ...,0,0,0,0.688525,0
7610,0,0,0,0,0,0,0,0,1,M1.94 [01:04 UTC]?5km S of Volcano Hawaii. htt...,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,1,1,0,0,0,1,10,67,1.412000,1.394000,8,8,1,7.250,s volcano hawaii,0,0,0,0.688525,0
7611,0,0,0,0,0,0,0,0,1,Police investigating after an e-bike collided ...,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,4,139,3.309186,2.496814,19,19,0,6.263,police investigating after ebike collided with...,0,0,0,0.688525,0


In [10]:
test_tweets.replace({True: 1, False: 0})

,keyword_0,keyword_1,keyword_2,keyword_3,keyword_4,keyword_5,keyword_6,keyword_7,keyword_8,text,continent_0,continent_1,continent_2,continent_3,continent_4,country_0,country_1,country_2,country_3,country_4,country_5,country_6,country_7,city_0,city_1,city_2,city_3,city_4,city_5,city_6,city_7,city_8,city_9,city_10,hasNum,hasLink,hasUser,hasHashtag,exclamacion,interrogacion,mayusculas,longitud,Puntaje_ser_1,Puntaje_ser_0,word_count,unique_word_count,url_count,mean_word_length,text_filtrado,contains_keyword,Tiene_key_impor,Tiene_key_no_impor,key_endoder,Label_key
0,0,0,0,0,0,0,0,0,1,Just happened a terrible car crash,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,36,1.706118,2.099882,6,6,0,4.833,just happened terrible car crash,0,0,0,0.688525,0
1,0,0,0,0,0,0,0,0,1,"Heard about #earthquake is different cities, s...",0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,1,66,1.512000,2.294000,9,9,0,6.222,heard about earthquake different cities stay s...,0,0,0,0.688525,0
2,0,0,0,0,0,0,0,0,1,"there is a forest fire at spot pond, geese are...",0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,98,1.928484,2.877516,19,19,0,4.105,there forest fire spot pond geese are fleeing ...,0,0,0,0.688525,0
3,0,0,0,0,0,0,0,0,1,Apocalypse lighting. #Spokane #wildfires,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,2,42,2.766851,4.039149,4,4,0,9.250,apocalypse lighting spokane wildfires,0,0,0,0.688525,0
4,0,0,0,0,0,0,0,0,1,Typhoon Soudelor kills 28 in China and Taiwan,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,4,47,1.439778,2.366222,8,8,0,4.750,typhoon soudelor kills in china and taiwan,0,0,0,0.688525,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3258,0,0,0,0,0,0,0,0,1,EARTHQUAKE SAFETY LOS ANGELES ÛÒ SAFETY FASTE...,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,45,57,1.412000,1.394000,8,7,0,6.000,earthquake safety los angeles safety faste...,0,0,0,0.688525,0
3259,0,0,0,0,0,0,0,0,1,Storm in RI worse than last hurricane. My city...,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,7,141,2.558703,3.247297,23,22,0,5.086,storm in ri worse than last hurricane my harde...,0,0,0,0.688525,0
3260,0,0,0,0,0,0,0,0,1,Green Line derailment in Chicago http://t.co/U...,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,9,57,1.652741,2.153259,6,6,1,8.333,green line derailment in chicago,0,0,0,0.688525,0
3261,0,0,0,0,0,0,0,0,1,MEG issues Hazardous Weather Outlook (HWO) htt...,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,1,1,0,0,0,0,15,67,1.412000,1.394000,7,7,1,8.428,meg issues hazardous weather outlook hwo,0,0,0,0.688525,0


In [11]:
test_tweets.columns

Index(['keyword_0', 'keyword_1', 'keyword_2', 'keyword_3', 'keyword_4',
       'keyword_5', 'keyword_6', 'keyword_7', 'keyword_8', 'text',
       'continent_0', 'continent_1', 'continent_2', 'continent_3',
       'continent_4', 'country_0', 'country_1', 'country_2', 'country_3',
       'country_4', 'country_5', 'country_6', 'country_7', 'city_0', 'city_1',
       'city_2', 'city_3', 'city_4', 'city_5', 'city_6', 'city_7', 'city_8',
       'city_9', 'city_10', 'hasNum', 'hasLink', 'hasUser', 'hasHashtag',
       'exclamacion', 'interrogacion', 'mayusculas', 'longitud',
       'Puntaje_ser_1', 'Puntaje_ser_0', 'word_count', 'unique_word_count',
       'url_count', 'mean_word_length', 'text_filtrado', 'contains_keyword',
       'Tiene_key_impor', 'Tiene_key_no_impor', 'key_endoder', 'Label_key'],
      dtype='object')

## Feature Engineering

In [12]:
def text_processing(tweet):
    
    #Generating the list of words in the tweet (hastags and other punctuations removed)
    def form_sentence(tweet):
        tweet_blob = TextBlob(tweet)
        return ' '.join(tweet_blob.words)
    new_tweet = form_sentence(tweet)
    
    #Removing stopwords and words with unusual symbols
    def no_user_alpha(tweet):
        tweet_list = [ele for ele in tweet.split() if ele != 'user']
        clean_tokens = [t for t in tweet_list if re.match(r'[^\W\d]*$', t)]
        clean_s = ' '.join(clean_tokens)
        clean_mess = [word for word in clean_s.split() if word.lower() not in stopwords.words('english')]
        return clean_mess
    no_punc_tweet = no_user_alpha(new_tweet)
    
    #Normalizing the words in tweets 
    def normalization(tweet_list):
        lem = WordNetLemmatizer()
        normalized_tweet = []
        for word in tweet_list:
            normalized_text = lem.lemmatize(word,'v')
            normalized_tweet.append(normalized_text)
        return normalized_tweet
    
    
    return normalization(no_punc_tweet)

In [13]:
#LIMPIO un poco
remove(train_tweets, test_tweets)

## Model Selection and Machine Learning

In [14]:
x = train_tweets
test = test_tweets

In [15]:
from sklearn.model_selection import train_test_split
msg_train, msg_test, label_train, label_test = train_test_split(x, y, test_size=0.2,random_state =87)

In [16]:
msg_train["text"]

3889    flattened early birds get de cups lag bout pun...
3051    contruction upgrading ferries earthquake stand...
6724    severe thunderstorm warning including russellv...
2302    charminar demolish in falling state anyway tak...
5859    cause you play like symphony play till your fi...
                              ...                        
5371    we just laughing and talking junk now everyone...
143          horrible accident man died in wings airplane
2747    in fragile global economy considering devastat...
333     sadly how windows reveals microsofts ethics ar...
3558    financialtimes ethiopian regimes continue rece...
Name: text, Length: 6090, dtype: object

# SOLO TEXT/TWEET

## MULTINOMIAL

In [49]:
#Machine Learning Pipeline
pipeline = Pipeline([
    #('bow',CountVectorizer(analyzer=text_processing,  ngram_range=(1, 1))),  # strings to token integer counts
    ('bow2',CountVectorizer(analyzer=text_processing,  ngram_range=(2, 3))),  # strings to token integer counts
    ('tfidf', TfidfTransformer(norm='l2')),  # integer counts to weighted TF-IDF scores
    ('classifier', MultinomialNB(alpha=1.3)),
])
pipeline.fit(msg_train["text"],label_train)
pred_train = pipeline.predict(msg_train["text"])
print(accuracy_score(pred_train,label_train))
pipeline.get_params

0.8893267651888341


<bound method Pipeline.get_params of Pipeline(memory=None,
         steps=[('bow2',
                 CountVectorizer(analyzer=<function text_processing at 0x000001CBF9E9F3A8>,
                                 binary=False, decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(2, 3), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, vocabulary=None)),
                ('tfidf',
                 TfidfTransformer(norm='l2', smooth_idf=True,
                                  sublinear_tf=False, use_idf=True)),
                ('classifier',
                 MultinomialNB(alpha=1.3, class_prior=None, fit_pri

In [50]:
predictions = pipeline.predict(msg_test["text"])

print(classification_report(predictions,label_test))
print ('\n')
print(confusion_matrix(predictions,label_test))
print(accuracy_score(predictions,label_test))

              precision    recall  f1-score   support

           0       0.92      0.76      0.83      1018
           1       0.64      0.86      0.73       505

    accuracy                           0.79      1523
   macro avg       0.78      0.81      0.78      1523
weighted avg       0.82      0.79      0.80      1523



[[773 245]
 [ 71 434]]
0.7925147734734077


In [51]:
predictions = pipeline.predict(test["text"])
predictions

array([1, 0, 1, ..., 1, 1, 1], dtype=int64)

In [53]:
comprobar(predictions)

F1 Score:  0.7906834201654919
F1 Score default:  0.7192766132346898


In [91]:
result = submission_output(ids, predictions,"Pipeline(tfidf y countvect (2,3) solo texto).csv")
result

,id,target
0,0,1
1,2,0
2,3,1
3,9,1
4,11,1
...,...,...
3258,10861,0
3259,10865,0
3260,10868,1
3261,10874,1


### LogisticRegression

In [21]:
from sklearn.linear_model import LogisticRegressionCV, Perceptron, LogisticRegression

#Machine Learning Pipeline
pipeline = Pipeline([
    #('bow',CountVectorizer(analyzer=text_processing,  ngram_range=(1, 1))),  # strings to token integer counts
    ('bow2',CountVectorizer(analyzer=text_processing,  ngram_range=(2, 3))),  # strings to token integer counts
    ('tfidf', TfidfTransformer(norm='l2')),  # integer counts to weighted TF-IDF scores
    ('classifier', LogisticRegression()),
])
pipeline.fit(msg_train["text"],label_train)

c:\users\hal9000\appdata\local\programs\python\python37\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Pipeline(memory=None,
         steps=[('bow2',
                 CountVectorizer(analyzer=<function text_processing at 0x000001CBE88FE798>,
                                 binary=False, decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(2, 3), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='(?u)\\b\\...
                ('tfidf',
                 TfidfTransformer(norm='l2', smooth_idf=True,
                                  sublinear_tf=False, use_idf=True)),
                ('classifier',
                 LogisticRegression(C=1.0, class_weight=None, dual=False,
                                    fit_intercept=True, intercept_scaling=1,
                            

In [22]:
predictions = pipeline.predict(msg_test["text"])

print(classification_report(predictions,label_test))
print ('\n')
print(confusion_matrix(predictions,label_test))
print(accuracy_score(predictions,label_test))

              precision    recall  f1-score   support

           0       0.89      0.77      0.83       974
           1       0.68      0.84      0.75       549

    accuracy                           0.80      1523
   macro avg       0.78      0.81      0.79      1523
weighted avg       0.82      0.80      0.80      1523



[[754 220]
 [ 90 459]]
0.7964543663821405


In [23]:
predictions = pipeline.predict(test["text"])

from sklearn.metrics import f1_score, recall_score, precision_score
comprobar(predictions)

F1 Score:  0.7912963530493411


In [60]:
result = submission_output(ids, predictions,"Pipeline con lr(tfidf y countvect (2,3) solo texto).csv")
result

,id,target
0,0,1
1,2,1
2,3,1
3,9,1
4,11,1
...,...,...
3258,10861,1
3259,10865,1
3260,10868,1
3261,10874,1


In [55]:
predictions

array([-5.40473453e-07,  9.71025119e-01,  6.73308668e-01, ...,
        1.00000002e+00, -2.39754773e-01,  6.21855317e-01])

In [ ]:
grid search bayesiano

### LogisticRegressionCV

In [237]:
#Machine Learning Pipeline
pipeline = Pipeline([
    #('bow',CountVectorizer(analyzer=text_processing,  ngram_range=(1, 1))),  # strings to token integer counts
    ('bow2',CountVectorizer(analyzer=text_processing,  ngram_range=(2, 3))),  # strings to token integer counts
    ('tfidf', TfidfTransformer(norm='l2')),  # integer counts to weighted TF-IDF scores
])
pipeline.fit(msg_train["text"],label_train)

Pipeline(memory=None,
         steps=[('bow2',
                 CountVectorizer(analyzer=<function text_processing at 0x0000018B8506DC18>,
                                 binary=False, decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(2, 3), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='(?u)\\b\\...
                 TfidfTransformer(norm='l2', smooth_idf=True,
                                  sublinear_tf=False, use_idf=True)),
                ('classifier',
                 LogisticRegressionCV(Cs=10, class_weight=None, cv=5,
                                      dual=False, fit_intercept=True,
                                      intercept_scaling=1.0, l1_r

In [238]:
predictions = pipeline.predict(msg_test["text"])

print(classification_report(predictions,label_test))
print ('\n')
print(confusion_matrix(predictions,label_test))
print(accuracy_score(predictions,label_test))

              precision    recall  f1-score   support

           0       0.86      0.79      0.82       927
           1       0.71      0.81      0.75       596

    accuracy                           0.79      1523
   macro avg       0.78      0.80      0.79      1523
weighted avg       0.80      0.79      0.80      1523



[[728 199]
 [116 480]]
0.7931713722915299


In [239]:
predictions = pipeline.predict(test["text"])

from sklearn.metrics import f1_score, recall_score, precision_score
comprobar(predictions)

F1 Score:  0.7916028194912658
F1 Score default:  0.7362296353762607


In [91]:
result = submission_output(ids, predictions,"Pipeline con lrCV(tfidf y countvect (2,3) solo texto).csv")
result

,id,target
0,0,1
1,2,1
2,3,1
3,9,1
4,11,1
...,...,...
3258,10861,1
3259,10865,1
3260,10868,1
3261,10874,1


### Perceptron

In [27]:
from sklearn.linear_model import LogisticRegressionCV, Perceptron, LogisticRegression

#Machine Learning Pipeline
pipeline = Pipeline([
    #('bow',CountVectorizer(analyzer=text_processing,  ngram_range=(1, 1))),  # strings to token integer counts
    ('bow2',CountVectorizer(analyzer=text_processing,  ngram_range=(2, 3))),  # strings to token integer counts
    ('tfidf', TfidfTransformer(norm='l2')),  # integer counts to weighted TF-IDF scores
    ('classifier', Perceptron()),
])
pipeline.fit(msg_train["text"],label_train)

Pipeline(memory=None,
         steps=[('bow2',
                 CountVectorizer(analyzer=<function text_processing at 0x000001CBE88FE798>,
                                 binary=False, decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(2, 3), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='(?u)\\b\\...
                ('tfidf',
                 TfidfTransformer(norm='l2', smooth_idf=True,
                                  sublinear_tf=False, use_idf=True)),
                ('classifier',
                 Perceptron(alpha=0.0001, class_weight=None,
                            early_stopping=False, eta0=1.0, fit_intercept=True,
                            max_iter=1

In [28]:
predictions = pipeline.predict(msg_test["text"])

print(classification_report(predictions,label_test))
print ('\n')
print(confusion_matrix(predictions,label_test))
print(accuracy_score(predictions,label_test))

              precision    recall  f1-score   support

           0       0.81      0.76      0.78       892
           1       0.69      0.74      0.71       631

    accuracy                           0.75      1523
   macro avg       0.75      0.75      0.75      1523
weighted avg       0.76      0.75      0.76      1523



[[681 211]
 [163 468]]
0.7544320420223244


In [29]:
predictions = pipeline.predict(test["text"])

from sklearn.metrics import f1_score, recall_score, precision_score
comprobar(predictions)

F1 Score:  0.7554397793441617


In [92]:
result = submission_output(ids, predictions,"Pipeline con Perceptron(tfidf y countvect (2,3) solo texto).csv")
result

,id,target
0,0,1
1,2,1
2,3,1
3,9,1
4,11,1
...,...,...
3258,10861,1
3259,10865,1
3260,10868,1
3261,10874,1


### XGBoost

In [33]:
#Machine Learning Pipeline
pipeline = Pipeline([
    #('bow',CountVectorizer(analyzer=text_processing,  ngram_range=(1, 1))),  # strings to token integer counts
    ('bow2',CountVectorizer(analyzer=text_processing,  ngram_range=(2, 3))),  # strings to token integer counts
    ('tfidf', TfidfTransformer(norm='l2')),  # integer counts to weighted TF-IDF scores
    ('classifier', XGBClassifier(n_estimators=1600,n_jobs=6,objective="binary:logistic",
                     max_depth= 60, learning_rate=0.01,subsample= 0.42,colsample_bytree=0.0933, gamma=1.0)),
])
pipeline.fit(msg_train["text"],label_train)
pred_train = pipeline.predict(msg_train["text"])
print(accuracy_score(pred_train,label_train))

0.8896551724137931


In [34]:
predictions = pipeline.predict(msg_test["text"])

print(classification_report(predictions,label_test))
print ('\n')
print(confusion_matrix(predictions,label_test))
print(accuracy_score(predictions,label_test))

              precision    recall  f1-score   support

           0       0.89      0.77      0.83       976
           1       0.67      0.83      0.74       547

    accuracy                           0.79      1523
   macro avg       0.78      0.80      0.78      1523
weighted avg       0.81      0.79      0.80      1523



[[752 224]
 [ 92 455]]
0.7925147734734077


In [35]:
predictions = pipeline.predict(test["text"])

from sklearn.metrics import f1_score, recall_score, precision_score
comprobar(predictions)

F1 Score:  0.7839411584431504


In [ ]:
result = submission_output(ids, predictions,"Pipeline con XGBoost(tfidf y countvect (2,3) solo texto).csv")
result

# TODO EL DF

### Columns

In [13]:
column_trans = ColumnTransformer(
    [('city_category', OneHotEncoder(dtype='int'),['keyword']),
    ('title_bow', CountVectorizer(analyzer=text_processing,  ngram_range=(2, 3)), 'text')],
    remainder='drop')

### Pipe y columns

In [17]:
col = list(train_tweets.columns)
col.remove("text")
col.remove('text_filtrado')
#col.remove("mean_word_length")
col

['keyword_0',
 'keyword_1',
 'keyword_2',
 'keyword_3',
 'keyword_4',
 'keyword_5',
 'keyword_6',
 'keyword_7',
 'keyword_8',
 'continent_0',
 'continent_1',
 'continent_2',
 'continent_3',
 'continent_4',
 'country_0',
 'country_1',
 'country_2',
 'country_3',
 'country_4',
 'country_5',
 'country_6',
 'country_7',
 'city_0',
 'city_1',
 'city_2',
 'city_3',
 'city_4',
 'city_5',
 'city_6',
 'city_7',
 'city_8',
 'city_9',
 'city_10',
 'hasNum',
 'hasLink',
 'hasUser',
 'hasHashtag',
 'exclamacion',
 'interrogacion',
 'mayusculas',
 'longitud',
 'Puntaje_ser_1',
 'Puntaje_ser_0',
 'word_count',
 'unique_word_count',
 'url_count',
 'mean_word_length',
 'contains_keyword',
 'Tiene_key_impor',
 'Tiene_key_no_impor',
 'key_endoder',
 'Label_key']

In [18]:
def Dejar(x):
    return x

In [19]:
#Machine Learning Pipeline

# define transformers
bow = CountVectorizer(analyzer="word", ngram_range=(1, 3))
bow1 = CountVectorizer(analyzer="char", ngram_range=(1, 3))
tfidf = TfidfTransformer(norm='l2', use_idf=True, smooth_idf=True, sublinear_tf=False,)
ohe = OneHotEncoder(handle_unknown='ignore')
bn = Binarizer()



# set up pipelines for each column group
key_pipe = Pipeline([
                        ('ohe', ohe),
                        #("bn",bn ),
                    ])
text_pipe = Pipeline([
                    ('bow', bow),
                    #('bow1', bow1), 
                    ('tfidf', tfidf),
                    ])
idem_pipe = Pipeline([
                    ('idem', FunctionTransformer(lambda x: x)), 
                    ])

# define column transformer and set n_jobs to use all cores
col_transformer = ColumnTransformer(
                    transformers=[
                        ('text', text_pipe, "text"),
#                        ('key', key_pipe, ["keyword"]),
                        ('Num',idem_pipe, col),
                    ],
                    remainder='drop',
                    n_jobs=-1
                    )

In [37]:
from xgboost import XGBClassifier

pipe = Pipeline([
            ("preprocessing", col_transformer),
            #('xgboost', XGBClassifier(n_estimators=900,n_jobs=6,objective="binary:logistic",
             #                         max_depth= 50, learning_rate=0.01,subsample= 0.45,colsample_bytree=0.1, gamma=1.1)),
            #('classifier', MultinomialNB(alpha=1.1))
            #('clf', SGDClassifier(loss='hinge', penalty='l2',alpha=1e-3, random_state=42,max_iter=5, tol=None)),
            #('classifier', Perceptron()),
            ('classifier', LogisticRegressionCV(n_jobs=4,Cs =25, max_iter=50,cv=10, solver ="liblinear", multi_class = "ovr",verbose =10))
            #('classifier', LogisticRegression()),
])

In [38]:
pipe.fit(msg_train,label_train)

pred_train = pipe.predict(msg_train)
print(accuracy_score(pred_train,label_train))

[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done   5 out of  10 | elapsed:  2.8min remaining:  2.8min
[Parallel(n_jobs=4)]: Done   7 out of  10 | elapsed:  2.9min remaining:  1.2min
[Parallel(n_jobs=4)]: Done  10 out of  10 | elapsed:  3.7min finished


[LibLinear]0.9866995073891626


In [39]:
predictions = pipe.predict(msg_test)

print(classification_report(predictions,label_test))
print ('\n')
print(confusion_matrix(predictions,label_test))
print(accuracy_score(predictions,label_test))

              precision    recall  f1-score   support

           0       0.85      0.80      0.83       900
           1       0.74      0.80      0.77       623

    accuracy                           0.80      1523
   macro avg       0.80      0.80      0.80      1523
weighted avg       0.81      0.80      0.80      1523



[[721 179]
 [123 500]]
0.8017071569271176


In [40]:
predictions = pipe.predict(test)

from sklearn.metrics import f1_score, recall_score, precision_score
comprobar(predictions)

F1 Score:  0.7992644805393809
F1 Score default:  0.7534813699661271


In [41]:
result = submission_output(ids, predictions,"Pipeline con L-RCV(tfidf y countvect (1,3),n_jobs=4,Cs =25,cv=10, solver =liblinear, multi_class = ovr,verbose =10,max_iter=100 ).csv")
result

,id,target
0,0,1
1,2,1
2,3,1
3,9,1
4,11,1
...,...,...
3258,10861,1
3259,10865,1
3260,10868,1
3261,10874,1


In [24]:
#Tunning LogisticRegressionCV

In [30]:
from sklearn.model_selection import GridSearchCV
parameters = {
    #"classifier__solver" : ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga']
    #"classifier__verbose": [1,2,10,20],
    "classifier__max_iter": [50,100,150,200,250,300,350,400,450],
    #"classifier__Cs": [15,20,25,30],
    #"classifier__class_weight":[None,"balanced"]
}

In [31]:
gs = GridSearchCV(pipe, parameters, cv=5, n_jobs=4,verbose=10)
b_m = gs.fit(msg_train,label_train)

pred_train = b_m.predict(msg_train)
print(accuracy_score(pred_train,label_train))

Fitting 5 folds for each of 9 candidates, totalling 45 fits


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done   5 tasks      | elapsed: 13.0min
[Parallel(n_jobs=4)]: Done  10 tasks      | elapsed: 24.7min
[Parallel(n_jobs=4)]: Done  17 tasks      | elapsed: 40.0min
[Parallel(n_jobs=4)]: Done  24 tasks      | elapsed: 57.2min
[Parallel(n_jobs=4)]: Done  33 tasks      | elapsed: 76.8min
[Parallel(n_jobs=4)]: Done  43 out of  45 | elapsed: 100.6min remaining:  4.7min
[Parallel(n_jobs=4)]: Done  45 out of  45 | elapsed: 101.5min finished
[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done   5 out of  10 | elapsed:  1.6min remaining:  1.6min
[Parallel(n_jobs=4)]: Done   7 out of  10 | elapsed:  1.7min remaining:   43.2s
[Parallel(n_jobs=4)]: Done  10 out of  10 | elapsed:  2.1min finished


[LibLinear]

c:\users\hal9000\appdata\local\programs\python\python37\lib\site-packages\sklearn\svm\base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


0.9860426929392446


In [32]:
b_m.best_params_

{'classifier__max_iter': 50}

In [33]:
predictions = b_m.predict(msg_test)

print(classification_report(predictions,label_test))
print ('\n')
print(confusion_matrix(predictions,label_test))
print(accuracy_score(predictions,label_test))

              precision    recall  f1-score   support

           0       0.85      0.80      0.83       899
           1       0.74      0.80      0.77       624

    accuracy                           0.80      1523
   macro avg       0.80      0.80      0.80      1523
weighted avg       0.81      0.80      0.80      1523



[[721 178]
 [123 501]]
0.8023637557452397


In [34]:
predictions = b_m.predict(test)

from sklearn.metrics import f1_score, recall_score, precision_score
comprobar(predictions)

F1 Score:  0.7998774134232303
F1 Score default:  0.7544189544941707


In [ ]:
#leve mejora

In [25]:
from sklearn.model_selection import GridSearchCV
parameters = {
    #"classifier__solver" : ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga']
    #"classifier__verbose": [1,2,10,20],
    "classifier__max_iter": [50,100,150,200,250,300,350,400,450],
    #"classifier__Cs": [15,20,25,30],
    #"classifier__class_weight":[None,"balanced"]
}

In [26]:
gs = GridSearchCV(pipe, parameters, cv=5, n_jobs=4,verbose=10)
b_m = gs.fit(msg_train,label_train)

pred_train = b_m.predict(msg_train)
print(accuracy_score(pred_train,label_train))

Fitting 5 folds for each of 32 candidates, totalling 160 fits


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done   5 tasks      | elapsed: 11.5min
[Parallel(n_jobs=4)]: Done  10 tasks      | elapsed: 17.2min
[Parallel(n_jobs=4)]: Done  17 tasks      | elapsed: 28.4min
[Parallel(n_jobs=4)]: Done  24 tasks      | elapsed: 37.0min
[Parallel(n_jobs=4)]: Done  33 tasks      | elapsed: 52.4min
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed: 66.3min
[Parallel(n_jobs=4)]: Done  53 tasks      | elapsed: 87.4min
[Parallel(n_jobs=4)]: Done  64 tasks      | elapsed: 109.4min
[Parallel(n_jobs=4)]: Done  77 tasks      | elapsed: 134.3min
[Parallel(n_jobs=4)]: Done  90 tasks      | elapsed: 163.4min
[Parallel(n_jobs=4)]: Done 105 tasks      | elapsed: 201.1min
[Parallel(n_jobs=4)]: Done 120 tasks      | elapsed: 238.3min
[Parallel(n_jobs=4)]: Done 137 tasks      | elapsed: 287.0min
[Parallel(n_jobs=4)]: Done 160 out of 160 | elapsed: 351.9min finished
[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 

[LibLinear]0.9661740558292282


In [27]:
b_m.best_params_

{'classifier__Cs': 30,
 'classifier__class_weight': None,
 'classifier__max_iter': 500}

In [28]:
predictions = b_m.predict(msg_test)

print(classification_report(predictions,label_test))
print ('\n')
print(confusion_matrix(predictions,label_test))
print(accuracy_score(predictions,label_test))

              precision    recall  f1-score   support

           0       0.86      0.80      0.83       904
           1       0.73      0.80      0.77       619

    accuracy                           0.80      1523
   macro avg       0.80      0.80      0.80      1523
weighted avg       0.81      0.80      0.80      1523



[[723 181]
 [121 498]]
0.8017071569271176


In [29]:
predictions = b_m.predict(test)

from sklearn.metrics import f1_score, recall_score, precision_score
comprobar(predictions)

F1 Score:  0.7980386147716826
F1 Score default:  0.7517890772128061


### NO  mejora

In [ ]:
#Tunning XGB

In [164]:
#gamma y subsample

In [165]:
from sklearn.model_selection import GridSearchCV
parameters = {
    'xgboost__gamma':[i/10 for i in range(20)],
    'xgboost__subsample':[0.5,0.1,0.15,0.2,0.25,0.3,0.35,0.4,0.45,0.5]
}

In [24]:
gs = GridSearchCV(pipe, parameters, cv=5, n_jobs=4,verbose=10)
b_m = gs.fit(msg_train,label_train)

Fitting 5 folds for each of 200 candidates, totalling 1000 fits


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done   5 tasks      | elapsed:  2.2min
[Parallel(n_jobs=4)]: Done  10 tasks      | elapsed:  3.0min
[Parallel(n_jobs=4)]: Done  17 tasks      | elapsed:  4.5min
[Parallel(n_jobs=4)]: Done  24 tasks      | elapsed:  6.1min
[Parallel(n_jobs=4)]: Done  33 tasks      | elapsed:  8.5min
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed: 11.1min
[Parallel(n_jobs=4)]: Done  53 tasks      | elapsed: 14.7min
[Parallel(n_jobs=4)]: Done  64 tasks      | elapsed: 17.1min
[Parallel(n_jobs=4)]: Done  77 tasks      | elapsed: 20.2min
[Parallel(n_jobs=4)]: Done  90 tasks      | elapsed: 23.7min
[Parallel(n_jobs=4)]: Done 105 tasks      | elapsed: 28.4min
[Parallel(n_jobs=4)]: Done 120 tasks      | elapsed: 31.2min
[Parallel(n_jobs=4)]: Done 137 tasks      | elapsed: 36.6min
[Parallel(n_jobs=4)]: Done 154 tasks      | elapsed: 41.3min
[Parallel(n_jobs=4)]: Done 173 tasks      | elapsed: 45.1min
[Parallel(

In [27]:
b_m.best_params_

{'xgboost__gamma': 1.1, 'xgboost__subsample': 0.45}

In [25]:
predictions = b_m.predict(msg_test)

print(classification_report(predictions,label_test))
print ('\n')
print(confusion_matrix(predictions,label_test))
print(accuracy_score(predictions,label_test))

              precision    recall  f1-score   support

           0       0.88      0.77      0.82       969
           1       0.67      0.82      0.74       554

    accuracy                           0.79      1523
   macro avg       0.78      0.80      0.78      1523
weighted avg       0.81      0.79      0.79      1523



[[746 223]
 [ 98 456]]
0.7892317793827971


In [26]:
predictions = b_m.predict(test)

from sklearn.metrics import f1_score, recall_score, precision_score
comprobar(predictions)

F1 Score:  0.7870058228623966
F1 Score default:  0.7221111555377848


In [ ]:
#n_estimators

In [36]:
from sklearn.model_selection import GridSearchCV
parameters = {
    'xgboost__n_estimators':[50*i for i in range(40)],
    #'xgboost__subsample':[0.5,0.1,0.15,0.2,0.25,0.3,0.35,0.4,0.45,0.5]
}

In [ ]:
gs = GridSearchCV(pipe, parameters, cv=5, n_jobs=4,verbose=10)
b_m = gs.fit(msg_train,label_train)

Fitting 5 folds for each of 40 candidates, totalling 200 fits


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done   5 tasks      | elapsed:  1.2min
[Parallel(n_jobs=4)]: Done  10 tasks      | elapsed:  1.8min
[Parallel(n_jobs=4)]: Done  17 tasks      | elapsed:  3.1min
[Parallel(n_jobs=4)]: Done  24 tasks      | elapsed:  3.9min
[Parallel(n_jobs=4)]: Done  33 tasks      | elapsed:  5.8min
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:  7.4min
[Parallel(n_jobs=4)]: Done  53 tasks      | elapsed:  9.7min
[Parallel(n_jobs=4)]: Done  64 tasks      | elapsed: 12.2min
[Parallel(n_jobs=4)]: Done  77 tasks      | elapsed: 15.6min
[Parallel(n_jobs=4)]: Done  90 tasks      | elapsed: 18.8min
[Parallel(n_jobs=4)]: Done 105 tasks      | elapsed: 23.4min


In [ ]:
b_m.best_params_

In [ ]:
predictions = b_m.predict(msg_test)

print(classification_report(predictions,label_test))
print ('\n')
print(confusion_matrix(predictions,label_test))
print(accuracy_score(predictions,label_test))

In [ ]:
predictions = b_m.predict(test)

from sklearn.metrics import f1_score, recall_score, precision_score
comprobar(predictions)